In [34]:
import importlib
from rl import multi_stock_env
importlib.reload(multi_stock_env)
MultiStockEnv = multi_stock_env.MultiStockEnv

In [2]:
from pprint import pprint
from pathlib import Path
import pandas as pd

In [3]:
MARKET = "csi300"
BENCHMARK = "SH000300"

EXP_NAME = "tutorial_exp"

data_provide_uri = "~/qlib_data/cn_data"

# Data

In [4]:
import qlib

qlib.init(provider_uri=data_provide_uri, region="cn")

[91441:MainThread](2025-08-19 23:13:24,282) INFO - qlib.Initialization - [config.py:452] - default_conf: client.
[91441:MainThread](2025-08-19 23:13:24,779) INFO - qlib.Initialization - [__init__.py:79] - qlib successfully initialized based on client settings.
[91441:MainThread](2025-08-19 23:13:24,779) INFO - qlib.Initialization - [__init__.py:81] - data_path={'__DEFAULT_FREQ': PosixPath('/root/qlib_data/cn_data')}


In [ ]:
import qlib
from qlib.data import D
import pandas as pd
from stable_baselines3 import PPO
  # 你需要自定义这个环境

# 初始化 Qlib
qlib.init(provider_uri=data_provide_uri, region="cn")
# 获取 csi300 成分股数据
instruments = D.list_instruments(D.instruments("csi300"))

results_df = {}
for instrument in list(instruments)[:10]:  # 只获取前10个股票数据，避免过多请求
    # 获取单个股票数据
    df = D.features(
        [instrument],
        ["$open", "$high", "$low", "$close", "$factor", "$adjclose", "$volume", "$change", "$amount"],
        start_time="2020-01-01",
        end_time="2025-08-31",
    )
    
    # 处理数据
    df = df.dropna()
    df = df.reset_index(drop=True)
    df.columns = [col.replace("$", "") for col in df.columns]

    if not df.empty:
        results_df[instrument] = df
    # 对 volume 做 Min-Max 归一化
    df["volume_norm"] = (df["volume"] - df["volume"].min()) / (df["volume"].max() - df["volume"].min())
    # 

#df = D.features(instruments, ["$close", "$open", "$high", "$low", "$volume"], start_time="2020-01-01", end_time="2025-12-31")



[91441:MainThread](2025-08-19 23:26:25,492) INFO - qlib.Initialization - [config.py:452] - default_conf: client.
[91441:MainThread](2025-08-19 23:26:25,494) INFO - qlib.Initialization - [__init__.py:79] - qlib successfully initialized based on client settings.
[91441:MainThread](2025-08-19 23:26:25,495) INFO - qlib.Initialization - [__init__.py:81] - data_path={'__DEFAULT_FREQ': PosixPath('/root/qlib_data/cn_data')}


In [ ]:
df = D.features(
        [instrument],
        ["$open", "$high", "$low", "$close", "$factor", "$adjclose", "$volume", "$change", "$amount"],
        start_time="2020-01-01",
        end_time="2025-08-31",
    )


$open     $high      $low    $close   $factor  \
instrument datetime                                                       
SZ000029   2020-01-02       NaN       NaN       NaN       NaN  0.262174   
           2020-01-03       NaN       NaN       NaN       NaN  0.262174   
           2020-01-06       NaN       NaN       NaN       NaN  0.262174   
           2020-01-07       NaN       NaN       NaN       NaN  0.262174   
           2020-01-08       NaN       NaN       NaN       NaN  0.262174   
...                         ...       ...       ...       ...       ...   
           2025-08-08  5.773397  5.848376  5.709526  5.820606  0.277701   
           2025-08-11  5.792529  6.009124  5.723107  5.956364  0.277686   
           2025-08-12  5.989833  6.456357  5.973172  6.270303  0.277693   
           2025-08-13  6.301220  6.381756  6.242901  6.334546  0.277709   
           2025-08-14  6.320358  6.450875  6.109309  6.164848  0.277696   

                       $adjclose        $volume   $change        $amount  
instrument datetime                                                       
SZ000029   2020-01-02        NaN            NaN       NaN            NaN  
           2020-01-03        NaN            NaN       NaN            NaN  
           2020-01-06        NaN            NaN       NaN            NaN  
           2020-01-07        NaN            NaN       NaN            NaN  
           2020-01-08        NaN            NaN       NaN            NaN  
...                          ...            ...       ...            ...  
           2025-08-08  48.020000  193063.984375  0.016489  111749.257812  
           2025-08-11  49.139999  278124.343750  0.023378  163317.203125  
           2025-08-12  51.730000  482204.718750  0.052681  300523.750000  
           2025-08-13  52.259998  323732.843750  0.010186  204183.734375  
           2025-08-14  50.860001  365683.187500 -0.026743  229850.890625  

[1362 rows x 9 columns]

In [27]:
results_df[next(iter(results_df))]

,open,high,low,close,factor,adjclose,volume,change,amount
0,11.144468,11.345270,11.077535,11.291723,0.669337,1841.680054,2.286189e+06,0.025532,2571196.500
1,11.338562,11.586217,11.325175,11.499203,0.669337,1875.520020,1.667614e+06,0.018376,1914495.500
2,11.385407,11.606288,11.318474,11.425567,0.669336,1863.510010,1.287968e+06,-0.006403,1477930.250
3,11.465767,11.566168,11.345286,11.479154,0.669338,1872.250000,1.088549e+06,0.004687,1247047.125
4,11.378771,11.412238,11.131115,11.151196,0.669339,1818.760010,1.266658e+06,-0.028571,1423608.750
...,...,...,...,...,...,...,...,...,...
1357,10.084078,10.124478,10.011355,10.019436,0.808019,1634.170044,1.026951e+06,-0.005613,1032586.750
1358,10.019428,10.035589,9.914386,9.938626,0.808018,1620.989990,1.156351e+06,-0.008065,1150305.125
1359,9.938604,10.019406,9.938604,9.962845,0.808017,1624.939941,8.416364e+05,0.002439,839635.625
1360,9.995197,10.019438,9.882074,9.906315,0.808019,1615.719971,1.385874e+06,-0.005677,1376777.875


In [ ]:
from rl.multi_stock_env import MultiStockEnv

In [ ]:
# 构建 RL 环境
env = MultiStockEnv(results_df, list(results_df))

# 训练 RL agent
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=100000)


In [ ]:

# 回测
obs = env.reset()
done = False
while not done:
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action)

# 评估策略表现
env.render()

In [ ]:
## TODO
## 需要对齐股价数据，保证每一行是同个时间点
## 用factor还原实际交易的价格
## 训练集和验证集要分开在两个时间段，做回测